In [1]:
%run trace_encode.ipynb

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c8c42cb8-d438-4266-b998-ddf54a1eba19;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 286ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/08 19:31:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/08 19:31:49 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
req_cols = ['file_path_ext','image_path_ext', 'parent_path_ext','object','action','timestamp_bins']

In [3]:
start_time = time.time()
df = de_dupe().cache()
print(df.count())
print("read in all data and count: " + str(time.time() - start_time))

 1:09AM UTC on Apr 06, 2023 --- read time: 4.475607395172119 seconds ---


3756864
read in all data and count: 32.920527935028076


In [10]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- file_path_ext: string (nullable = true)



In [5]:
start_time = time.time()
df.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/dedupe")

In [6]:
df.unpersist()

DataFrame[id: string, timestamp: timestamp, objectID: string, actorID: string, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, event_minute: int, event_day: int, event_hour: int, malicious: int, file_path_ext: string]

In [3]:
start_time = time.time()
df = spark.read.parquet(f"{s3_url_trusted}/prod/graph/dedupe").cache()

In [4]:
print(df.count())
print("write, read, and count: " + str(time.time() - start_time))

3756864
write, read, and count: 50.82806396484375


In [17]:
%run trace_encode.ipynb

In [4]:
#now create graph with motif 2, create bins that you will use for all ts diff, and bin
#create two motifs at a time, join, unpersist, write, then unpersist
start_time = time.time()
df_2 = create_graph2(df).cache()
print("graph: "+ str(time.time() - start_time))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 0.42069172859191895
filtered connections: 0.4339885711669922
graph: 0.6944127082824707


In [5]:
start_time = time.time()
print(df_2.count())
print("count: "+ str(time.time() - start_time))

30613703
count: 30.060638189315796


In [11]:
#create ts_bins that you will use for all subsequent binning
'''start_time = time.time()
df_2 = ts_diff(df_2).cache()
bins = df_2.approxQuantile("timestamp_difference", [0.2, 0.4, .6, .8], 0)
print(bins)
print("binning: "+ str(time.time() - start_time))'''

[1730209.0001106262, 8254118.999958038, 21626879.999876022, 44756893.000125885]
binning: 250.15912413597107


In [6]:
start_time = time.time()
#explode
df_2 = transp_expl2(df_2).cache()
df_2 = df_2.withColumn('motif',lit(2)).cache()
print("create graph and explode: "+ str(time.time() - start_time))
df_2 = bin_it(df_2).cache()
print("bin time: "+ str(time.time() - start_time))

create graph and explode: 0.3021578788757324
bin time: 0.6034109592437744


In [12]:
df_2.printSchema()

root
 |-- Trace: string (nullable = false)
 |-- event: string (nullable = false)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- file_path_ext: string (nullable = true)
 |-- motif: integer (nullable = false)
 |-- timestamp_bins: double (nullable = true)



In [7]:
#write to parquet and unpersist
start_time = time.time()
df_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/2")
print("write time: "+ str(time.time() - start_time))

write time: 421.1579158306122


In [8]:
df_2.unpersist()

DataFrame[Trace: string, event: string, src: string, dst: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, malicious: int, file_path_ext: string, motif: int, timestamp_bins: double]

In [9]:
#now df3
start_time = time.time()
df_3 = create_graph3(df).cache()
print("graph: "+ str(time.time() - start_time))

found connections: 0.12255430221557617
graph: 0.26387548446655273


In [10]:
start_time = time.time()
print(df_3.count())
print("count: "+ str(time.time() - start_time))

110105995
count: 194.46526455879211


In [11]:
start_time = time.time()
df_3 = transp_expl3(df_3).cache()
df_3 = df_3.withColumn('motif',lit(3)).cache()
print("transp-explode: "+ str(time.time() - start_time))
df_3 = bin_it(df_3).cache()
print("binned: "+ str(time.time() - start_time))

transp-explode: 0.16888213157653809
binned: 0.29276418685913086


In [12]:
#write to parquet and unpersist
start_time = time.time()
df_3.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/3")
print("write time (np): "+ str(time.time() - start_time))

write time (np): 1485.1701052188873


In [13]:
df_3.unpersist()

DataFrame[Trace: string, event: string, src: string, dst: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, malicious: int, file_path_ext: string, motif: int, timestamp_bins: double]

In [6]:
#now df4
start_time = time.time()
df_4 = create_graph4(df).cache()
print("graph: "+ str(time.time() - start_time))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 0.5519614219665527
graph: 0.9262793064117432


In [7]:
start_time = time.time()
print(df_4.count())
print("count: "+ str(time.time() - start_time))

109881292
count: 222.0687873363495


In [8]:
start_time = time.time()
df_4 = transp_expl4(df_4).cache()
df_4 = df_4.withColumn('motif',lit(4)).cache()
print("transp-explode: "+ str(time.time() - start_time))
df_4 = bin_it(df_4).cache()
print("binned: "+ str(time.time() - start_time))

transp-explode: 0.3355429172515869
binned: 0.6755263805389404


In [9]:
#write to parquet and unpersist
start_time = time.time()
df_4.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/4")
print("write time: "+ str(time.time() - start_time))

write time: 1909.8718740940094


In [10]:
df_4.unpersist()

DataFrame[Trace: string, event: string, src: string, dst: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, malicious: int, file_path_ext: string, motif: int, timestamp_bins: double]

In [11]:
#now df6
start_time = time.time()
df_6 = create_graph6(df).cache()
print("graph: "+ str(time.time() - start_time))

found connections: 0.24536418914794922
graph: 0.4464895725250244


In [12]:
start_time = time.time()
print(df_6.count())
print("count: "+ str(time.time() - start_time))

127702239
count: 515.6148951053619


In [13]:
start_time = time.time()
df_6 = transp_expl(df_6).cache()
df_6 = df_6.withColumn('motif',lit(6)).cache()
print("transp-explode: "+ str(time.time() - start_time))
df_6 = bin_it(df_6).cache()
print("binned: "+ str(time.time() - start_time))

transp-explode: 0.26151442527770996
binned: 0.3914022445678711


In [14]:
#write to parquet and unpersist
start_time = time.time()
df_6.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/6")
print("write time: "+ str(time.time() - start_time))

write time: 3273.2002720832825


In [5]:
#now df5
start_time = time.time()
df_5 = create_graph5(df).cache()
print("create graph: "+ str(time.time() - start_time))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


found connections: 0.6179800033569336
create graph: 1.0455591678619385


In [15]:
df_6.unpersist()

DataFrame[Trace: string, event: string, src: string, dst: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, malicious: int, file_path_ext: string, motif: int, timestamp_bins: double]

In [6]:
start_time = time.time()
print(df_5.count())
print("count: "+ str(time.time() - start_time))

97166164
count: 550.1845195293427


In [7]:
start_time = time.time()
df_5 = transp_expl5(df_5).cache()
df_5 = df_5.withColumn('motif',lit(5)).cache()
print("transp-explode: "+ str(time.time() - start_time))
df_5 = bin_it(df_5).cache()
print("binned: "+ str(time.time() - start_time))

transp-explode: 0.37686634063720703
binned: 0.8165910243988037


In [8]:
#write to parquet and unpersist
start_time = time.time()
df_5.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/5")
print("write time: "+ str(time.time() - start_time))

write time: 2679.5069410800934


In [9]:
df_5.unpersist()

DataFrame[Trace: string, event: string, src: string, dst: string, timestamp: timestamp, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, malicious: int, file_path_ext: string, motif: int, timestamp_bins: double]

In [21]:
df.unpersist()

DataFrame[id: string, timestamp: timestamp, objectID: string, actorID: string, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, event_minute: int, event_day: int, event_hour: int, malicious: int, file_path_ext: string]

In [17]:
#read in and count all motifs
start_time = time.time()
df_all_motifs = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motifs/*").cache()
print("read time all: "+ str(time.time() - start_time))

read time all: 1.114145040512085


In [18]:
start_time = time.time()
#now count everything you read in
print(df_all_motifs.count())
print("read time all: "+ str(time.time() - start_time))

ERROR:root:KeyboardInterrupt while sending command.             (32 + 32) / 616]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

### now run all motifs through one hot and see what you get


In [7]:
df_all_motifs.printSchema()

root
 |-- Trace: string (nullable = true)
 |-- event: string (nullable = true)
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- malicious: integer (nullable = true)
 |-- file_path_ext: string (nullable = true)
 |-- motif: integer (nullable = true)
 |-- timestamp_bins: double (nullable = true)



In [9]:
df_all_motifs.limit(6).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Trace,event,src,dst,timestamp,object,action,hostname,user_name,privileges,...,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,malicious,file_path_ext,motif,timestamp_bins
0,100236-2,e1,12541710-ae34-495d-bac2-9fde87b1e4e0,65fdc95d-207b-4a4e-a630-2bf056b0d762,2019-09-23 14:27:16.609,FILE,READ,SysClient0419.systemia.com,None,None,...,None,\Device\HarddiskVolume1\install\MarkupSafe\bui...,None,None,None,None,0,empty,2,NaN
1,100236-2,e2,65fdc95d-207b-4a4e-a630-2bf056b0d762,9fb3404f-50d8-4a42-b479-89b5d4e84f1f,2019-09-23 15:05:03.986,PROCESS,OPEN,SysClient0419.systemia.com,None,None,...,None,None,None,None,None,None,0,None,2,1.0
2,100246-2,e1,61d0949f-cd87-439f-ac78-f2a8990596b6,65fdc95d-207b-4a4e-a630-2bf056b0d762,2019-09-23 14:54:39.243,FILE,READ,SysClient0419.systemia.com,None,None,...,None,\Device\HarddiskVolume1\Users\khendricks\AppDa...,None,None,None,None,0,empty,2,NaN
3,100246-2,e2,65fdc95d-207b-4a4e-a630-2bf056b0d762,9fb3404f-50d8-4a42-b479-89b5d4e84f1f,2019-09-23 15:05:03.986,PROCESS,OPEN,SysClient0419.systemia.com,None,None,...,None,None,None,None,None,None,0,None,2,0.0
4,100315-2,e1,2905ddbe-cfb8-4792-9bac-53940ae91234,65fdc95d-207b-4a4e-a630-2bf056b0d762,2019-09-23 14:27:26.440,FILE,READ,SysClient0419.systemia.com,None,None,...,None,\Device\HarddiskVolume1\ncr\mantra\runtime\Lib...,None,None,None,None,0,empty,2,NaN
5,100315-2,e2,65fdc95d-207b-4a4e-a630-2bf056b0d762,9fb3404f-50d8-4a42-b479-89b5d4e84f1f,2019-09-23 15:05:03.986,PROCESS,OPEN,SysClient0419.systemia.com,None,None,...,None,None,None,None,None,None,0,None,2,1.0


### I am now going to reduce the rows. Should this be a function, absolutely. Maybe later. 

#### Start with pre-processing for overall df. 

In [6]:
#so now pivot, groupBy Trace and return first motif, src and destination
#first pivot aka transpose and keep all events
df_all_motifs = df_all_motifs.groupBy('Trace').pivot('event')\
.agg(first('src'),first('dst'), first('motif')).cache()

In [7]:
#consolidate motif columns by finding the average or mode of each column
df_all_motifs = df_all_motifs.withColumn('motif', col('e1_first(motif)')).cache()

In [13]:
df_all_motifs_m.limit(2).toPandas()

,Trace,e1_first(src),e1_first(dst),e1_first(motif),e2_first(src),e2_first(dst),e2_first(motif),e3_first(src),e3_first(dst),e3_first(motif),e4_first(src),e4_first(dst),e4_first(motif),e5_first(src),e5_first(dst),e5_first(motif),e6_first(src),e6_first(dst),e6_first(motif),motif
0,100016-2,944be8ff-30c7-46cf-841f-ba87a391f69f,65fdc95d-207b-4a4e-a630-2bf056b0d762,2,65fdc95d-207b-4a4e-a630-2bf056b0d762,9fb3404f-50d8-4a42-b479-89b5d4e84f1f,2,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,2
1,100074-5,e2bc274c-4bd6-411f-afcd-0353b8f9963e,fa89e9b4-2776-4ca2-8555-0c649b0e1f55,5,fa89e9b4-2776-4ca2-8555-0c649b0e1f55,273bfab2-2849-42d2-bf5e-7f01d185c792,5,273bfab2-2849-42d2-bf5e-7f01d185c792,d952da47-3aa0-41ca-87e8-94be47f78234,5.0,d952da47-3aa0-41ca-87e8-94be47f78234,3b587487-ead0-4209-a4b6-ad6561a96bfa,5.0,3b587487-ead0-4209-a4b6-ad6561a96bfa,14ac57f8-46ff-422b-bc8d-1f947099ee4c,5.0,None,None,NaN,5


In [8]:
#drop rendundat motif columns
df_all_motifs = df_all_motifs.drop('e1_first(motif)','e2_first(motif)','e3_first(motif)',
                       'e4_first(motif)','e5_first(motif)','e6_first(motif)').cache()

In [9]:
#drop redundant src columns and keep the destination
df_all_motifs = df_all_motifs.drop('e2_first(src)','e3_first(src)','e4_first(src)',
                       'e5_first(src)','e6_first(src)').cache()

In [10]:
#write to parquet and unpersist
start_time = time.time()
df_all_motifs.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivoted")
print("write time: "+ str(time.time() - start_time))

write time: 5085.627703428268


In [11]:
df_all_motifs.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, e4_first(dst): string, e5_first(dst): string, e6_first(dst): string, motif: int]

In [3]:
#read in and count all motifs
start_time = time.time()
df_all_motifs_piv = spark.read.parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivoted").cache()
print("read time all: "+ str(time.time() - start_time))

read time all: 6.313919544219971


In [4]:
#read it back in
start_time = time.time()
print(start_time)
#now count everything you read in
print(df_all_motifs_piv.count())
print("read time all: "+ str(time.time() - start_time))

1681010386.8307817


475469393
read time all: 168.15877604484558


#### Now let's start with trace length 6

In [1]:
%run trace_encode.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-84576547-9e25-4bf9-9f4d-a508eb00dabb;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 146ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/09 03:18:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 03:18:36 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"

In [5]:
#algorithm as follows (repeat for all trace length 6 through 3: 
#create int df, drop last column 
df_int_6 = df_all_motifs_piv.drop('e6_first(dst)').cache()

In [6]:
start = time.time()
print(df_int_6.count())
print("int 6 count: "+ str(time.time() - start))

475469393
int 6 count: 231.39778804779053


In [9]:
'''#write pivint6 now
start_time = time.time()
print(start_time)
df_int_6.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint6")
print("write time: "+ str(time.time() - start_time))'''

'#write pivint6 now\nstart_time = time.time()\nprint(start_time)\ndf_int_6.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint6")\nprint("write time: "+ str(time.time() - start_time))'

In [7]:
df_all_motifs_piv.unpersist()
#df_int_6.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, e4_first(dst): string, e5_first(dst): string, e6_first(dst): string, motif: int]

In [ ]:
'''#read in pivint6
#I think I should read in the partial first.
start_time = time.time()
print(start_time)
df_int_6 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint6").cache()
print("read time all: "+ str(time.time() - start_time))'''

In [ ]:
'''start = time.time()
print(df_int_6.count())
print("int 6 count: "+ str(time.time() - start))'''

In [8]:
start = time.time()
#concat columns and compare to drop
df_int_6 = df_int_6.withColumn('concat',concat(df_int_6.columns[1], df_int_6.columns[2], 
                                     df_int_6.columns[3], df_int_6.columns[4],
                                                            df_int_6.columns[5],df_int_6.columns[6])).cache()
print("int 6 concat: "+ str(time.time() - start))

int 6 concat: 0.07466006278991699


In [9]:
start = time.time()
print(df_int_6.count())
print("count: "+ str(time.time() - start))

475469393
count: 205.18920969963074


In [19]:
df_int_6.limit(2).toPandas()

,Trace,e1_first(src),e1_first(dst),e2_first(dst),e3_first(dst),e4_first(dst),e5_first(dst),motif,concat
0,1425929187402-5,158f7657-c0dd-4541-9ac4-6cf3a9e987f7,bf963665-a321-47e8-88b9-f4e9b9a13ac9,f3f37b3e-5565-48f5-ae3a-4c9583110734,c38cd8e6-553f-4814-898f-3d83eaf2b447,de4be40d-15b3-465a-8dc8-d18655de613e,201719d5-c2ed-43c6-be16-649c8374b14e,5,158f7657-c0dd-4541-9ac4-6cf3a9e987f7bf963665-a...
1,1425929187412-6,f89d404f-3530-4d25-b9b0-327b8f8c8b37,bf963665-a321-47e8-88b9-f4e9b9a13ac9,77a17f8a-1cdc-44e8-ac50-43ae85e69ce8,c38cd8e6-553f-4814-898f-3d83eaf2b447,de4be40d-15b3-465a-8dc8-d18655de613e,f3f37b3e-5565-48f5-ae3a-4c9583110734,6,f89d404f-3530-4d25-b9b0-327b8f8c8b37bf963665-a...


In [18]:
start = time.time()
print(start)
#now set the window. partition by concat and order by concat all for int mat, pull inv filter
#if this step is too hot to handle then you can revert back to simply select columns etc
window = Window.partitionBy("concat").orderBy(col("motif").desc())
col_int = df_int_6.orderBy(col('concat'))\
            .withColumn("rank", rank().over(window))\
                .filter((col("rank") != 1) & (col('concat').isNotNull()))\
                .select('Trace').distinct().collect()
print("int 6 col gen: "+ str(time.time() - start))

1680969408.7436364


int 6 col gen: 441.4695794582367


In [27]:
print([r.Trace for r in col_int[0:5]])

['661425149970-5', '300647924572-5', '146029139753-5', '154619094741-5', '1425929184918-5']


In [28]:
#create df for anti-join
start = time.time()
print(start)
#turn the col_int into a dataframe
from pyspark.sql import Row
R = Row('Trace')
# use the row function to create a bunch of rows
df_int = spark.createDataFrame([R(x.Trace) for x in col_int]).cache()
print("dataframe create: "+ str(time.time() - start))

1680970628.46869
dataframe create: 223.79657220840454


In [29]:
#count for df_int
start = time.time()
print(start)
print(df_int.count())
print(len(col_int[0][:]))
print("df_int count: "+ str(time.time() - start))

1680970852.2700748


17924097
1
df_int count: 2.3876726627349854


In [30]:
#write the int dataframe right now
start_time = time.time()
print(start_time)
df_int.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/filter_list")
print("write time: "+ str(time.time() - start_time))

1680970854.6652005


write time: 33.91192293167114


In [32]:
#clear col_int
df_int.unpersist()
col_int.clear()

ConnectionRefusedError: [Errno 111] Connection refused

In [17]:
#now read in filter list, and then read in motif 5 data
start_time = time.time()
print(start_time)
df_filter = spark.read.parquet(f"{s3_url_trusted}/prod/graph/filter_list").cache()
print("read time all: "+ str(time.time() - start_time))

1681012096.8483539
read time all: 0.4086313247680664


In [18]:
#count filter
start = time.time()
print(start)
print(df_filter.count())
print("df_int count: "+ str(time.time() - start))

1681012100.764311


17924097
df_int count: 1.9081246852874756


In [12]:
#try to filter motif 5, if not we may have an issue
#read in motif 5
start_time = time.time()
print(start_time)
df_motif_5 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motifs/5").cache()
print("read time all: "+ str(time.time() - start_time))

1680985633.1623528
read time all: 0.45999789237976074


In [13]:
#count motif 5
start = time.time()
print(start)
print(df_motif_5.count())
print("motif 5 count: "+ str(time.time() - start))

1680985633.627209


485830820
motif 5 count: 214.52204751968384


In [7]:
df_filter.limit(2).toPandas()

,Trace
0,257698242047-5
1,1666447685536-5


In [24]:
#df_motif_5.where(col('Trace') == '1666447685536-5').show()

+---------------+-----+--------------------+--------------------+--------------------+-------+------+--------------------+---------+----------+----------------+-----------------+--------+---------+---------+--------+-----------------+---------------+---------+-------------+-----+--------------+
|          Trace|event|                 src|                 dst|           timestamp| object|action|            hostname|user_name|privileges|      image_path|parent_image_path|new_path|file_path|direction|logon_id|requesting_domain|requesting_user|malicious|file_path_ext|motif|timestamp_bins|
+---------------+-----+--------------------+--------------------+--------------------+-------+------+--------------------+---------+----------+----------------+-----------------+--------+---------+---------+--------+-----------------+---------------+---------+-------------+-----+--------------+
|1666447685536-5|   e1|d52980c4-b337-474...|f69acad0-defb-449...|2019-09-24 15:44:...|PROCESS|  OPEN|SysClient08

In [14]:
#filter motif 5
start = time.time()
print(start)
df_motif_5 = df_motif_5.join(df_filter, ["Trace"], "leftanti").cache()
print("df_motif_5 join: "+ str(time.time() - start))

1680985848.1536875
df_motif_5 join: 0.04090309143066406


In [15]:
#count motif 5
start = time.time()
print(start)
print(df_motif_5.count())
print("motif 5 count: "+ str(time.time() - start))

1680985848.1987429


396210335
motif 5 count: 349.82933235168457


In [16]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
df_motif_5.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/filtered/5")
print("write time: "+ str(time.time() - start_time))

1680986198.0326545


write time: 918.3402369022369


In [11]:
#before you read in df_5 you need to filter the df_int matrix
#first separate the int dfs into motifs - 1 and all other motifs
df_int_a = df_int_6.filter(col("motif") == 5).cache()

In [12]:
#count df_int_a
start = time.time()
print(start)
print(df_int_a.count())
print("df_int_a count: "+ str(time.time() - start))

1681011751.023592


97166164
df_int_a count: 115.9735631942749


In [13]:
df_int_b = df_int_6.filter(col("motif") < 5).cache()

In [14]:
#count df_int_b
start = time.time()
print(start)
print(df_int_b.count())
print("df_int_a count: "+ str(time.time() - start))

1681011867.0215585


250600990
df_int_a count: 135.9872283935547


In [15]:
df_int_6.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, e4_first(dst): string, e5_first(dst): string, motif: int, concat: string]

In [19]:
#filter int_a
#now execute anti join
start = time.time()
print(start)
df_int_a = df_int_a.join(df_filter, ["Trace"], "leftanti")
print("antijoin: "+ str(time.time() - start))

1681012117.6044633
antijoin: 0.02583456039428711


In [20]:
#count a 
#count df_int_a
start = time.time()
print(start)
print(df_int_a.count())
print("df_int_a count: "+ str(time.time() - start))

1681012122.1516697


79242067
df_int_a count: 20.231421947479248


In [23]:
#union all the ints, select on motif 
df_int_5 = df_int_a.union(df_int_b)

In [24]:
start_time = time.time()
df_int_5.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint5")
print("write time: "+ str(time.time() - start_time))

write time: 1048.8404083251953


In [ ]:
#

In [ ]:
'''#read in motif 5
start_time = time.time()
print(start_time)
df_motif_5 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motifs/5").cache()
print("read time all: "+ str(time.time() - start_time))'''

In [ ]:
'''#count motif 5
start = time.time()
print(start)
print(df_motif_5.count())
print("df_int count: "+ str(time.time() - start))'''

In [ ]:
'''#read in and count all motifs
start_time = time.time()
df_all_motifs_piv = spark.read.parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivoted").cache()
print("read time all: "+ str(time.time() - start_time))'''

In [ ]:
'''#read it back in
start_time = time.time()
#now count everything you read in
print(df_all_motifs_piv.count())
print("read time all: "+ str(time.time() - start_time))'''

In [ ]:
'''start = time.time()
#now filter big matrix and write it
df_all_motifs_piv = df_all_motifs_piv.filter(~col("Trace").isin(inv_filt))
print("all filter: "+ str(time.time() - start))'''

In [ ]:
'''start_time = time.time()
df_all_motifs_piv.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all_motifs/piv6")
print("write time: "+ str(time.time() - start_time))'''

#### Now run for 5, beginning by reading everything back in

In [1]:
%run trace_encode.ipynb

Matplotlib is building the font cache; this may take a moment.


:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a2154af2-4472-430c-9963-9a44e71528c9;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (41ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (32ms)
:: resolution report :: resolve 703ms :: artifacts dl 74ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/04/09 14:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 14:07:57 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"

In [3]:
#I think I should read in the partial first.
start_time = time.time()
print(start_time)
df_int_5 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint/pivint5").cache()
print("read time all: "+ str(time.time() - start_time))

1681049287.957085


read time all: 8.510698318481445


In [4]:
start_time = time.time()
print(start_time)
#now count everything you read in
print(df_int_5.count())
print("read time all: "+ str(time.time() - start_time))

1681049305.9767997


329843057
read time all: 134.19801354408264


In [11]:
#quick peek at what the data looks like 
df_int_5.limit(2).toPandas()

,Trace,e1_first(src),e1_first(dst),e2_first(dst),e3_first(dst),e4_first(dst),motif,concat
0,1005022347378-5,91665c33-9ceb-4f5b-955e-e6fae3e70863,6257db09-fa0f-4160-bc17-97f22adc951c,a6e8c449-1e46-4092-afc9-4b88b2590bd0,add1b4c9-690e-407a-af00-6bf83e5ce7a8,7a0da822-d7fd-4705-9119-43a42d24f56e,5,91665c33-9ceb-4f5b-955e-e6fae3e708636257db09-f...
1,1005022347714-5,97b06ce8-279f-4ef7-ba55-1f61f2efbf3f,cbd63108-5c09-4274-951d-b3bacbc2a696,c48e49bb-7819-4726-a0a3-6dd67e0d361a,add1b4c9-690e-407a-af00-6bf83e5ce7a8,7a0da822-d7fd-4705-9119-43a42d24f56e,5,97b06ce8-279f-4ef7-ba55-1f61f2efbf3fcbd63108-5...


In [8]:
df_int_5.select('motif').distinct().show()

+-----+
|motif|
+-----+
|    5|
|    3|
|    4|
|    2|
+-----+



In [9]:
#first drop the last column and the current motif
df_int_5 = df_int_5.drop('e5_first(dst)').cache()

In [12]:
start = time.time()
print(start)
#concat columns and compare to drop
df_int_5 = df_int_5.withColumn('concat',concat(df_int_5.columns[1], df_int_5.columns[2], 
                                     df_int_5.columns[3], df_int_5.columns[4],df_int_5.columns[5])).cache()
print("int 5 concat: "+ str(time.time() - start))

1681049688.5393791
int 5 concat: 0.037262916564941406


In [13]:
start_time = time.time()
#now count everything you read in
print(df_int_5.count())
print("read time all: "+ str(time.time() - start_time))

329843057
read time all: 967.8283665180206


In [14]:
start = time.time()
print(start)
#now set the window. partition by concat and order by concat all for int mat, pull inv filter
#if this step is too hot to handle then you can revert back to simply select columns etc
window = Window.partitionBy("concat").orderBy(col("motif").desc())
col_int = df_int_5.orderBy(col('concat'))\
            .withColumn("rank", rank().over(window))\
                .filter((col("rank") != 1) & (col('concat').isNotNull()))\
                .select('Trace').distinct().collect()
print("int 5 col gen: "+ str(time.time() - start))

1681050661.6851473


int 5 col gen: 691.7900104522705


In [15]:
#create df for anti-join
start = time.time()
print(start)
#turn the col_int into a dataframe
from pyspark.sql import Row
R = Row('Trace')
# use the row function to create a bunch of rows
df_filter = spark.createDataFrame([R(x.Trace) for x in col_int]).cache()
print("dataframe create: "+ str(time.time() - start))

1681051353.4798877
dataframe create: 153.89524483680725


In [ ]:
#count for df_int
start = time.time()
print(start)
print(df_filter.count())
print(len(col_int[0]))
print("df_int count: "+ str(time.time() - start))

In [16]:
#write the int dataframe right now
start_time = time.time()
print(start_time)
df_filter.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/filters_all/filter_5")
print("write time: "+ str(time.time() - start_time))

1681051507.379963


write time: 36.7403998374939


In [17]:
#clear col_int
df_filter.unpersist()
col_int.clear()

In [18]:
#now read in filter list, and then read in motif 5 data
start_time = time.time()
print(start_time)
df_filter = spark.read.parquet(f"{s3_url_trusted}/prod/graph/filters_all/filter_5").cache()
print("read time all: "+ str(time.time() - start_time))

1681051559.6483438
read time all: 0.22326350212097168


In [19]:
#count filter
start = time.time()
print(start)
print(df_filter.count())
print("df_int count: "+ str(time.time() - start))

1681051564.0875297


12493038
df_int count: 1.69136381149292


##### now filter pivint, then filter motif 4

In [20]:
#before you read in df_5 you need to filter the df_int matrix
#first separate the int dfs into motifs - 1 and all other motifs
df_int_a = df_int_5.filter(col("motif") == 4).cache()

In [21]:
#count df_int_a
start = time.time()
print(start)
print(df_int_a.count())
print("df_int_a count: "+ str(time.time() - start))

1681051581.0121644


109881292
df_int_a count: 121.39162993431091


In [22]:
df_int_b = df_int_5.filter(col("motif") < 4).cache()

In [23]:
#count df_int_b
start = time.time()
print(start)
print(df_int_b.count())
print("df_int_a count: "+ str(time.time() - start))

1681051702.4279013


140719698
df_int_a count: 79.7126522064209


In [24]:
df_int_5.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, e4_first(dst): string, motif: int, concat: string]

In [26]:
#filter int_a
#now execute anti join
start = time.time()
print(start)
df_int_a = df_int_a.join(df_filter, ["Trace"], "leftanti").cache()
print("antijoin: "+ str(time.time() - start))

1681051782.2638674
antijoin: 0.02251887321472168


In [27]:
#count a 
#count df_int_a
start = time.time()
print(start)
print(df_int_a.count())
print("df_int_a count: "+ str(time.time() - start))

1681051782.2904072


97388254
df_int_a count: 134.08898544311523


In [28]:
#union all the ints, select on motif 
df_int_4 = df_int_a.union(df_int_b).cache()

In [29]:
#count df_int_a
start = time.time()
print(start)
print(df_int_4.count())
print("df_int_4 count: "+ str(time.time() - start))

1681051916.431659


238107952
df_int_4 count: 162.86321139335632


In [30]:
df_int_a.unpersist()
df_int_b.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, e4_first(dst): string, motif: int, concat: string]

In [31]:
start_time = time.time()
df_int_4.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint/pivint4")
print("write time: "+ str(time.time() - start_time))

write time: 770.2307872772217


In [32]:
df_int_4.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, e4_first(dst): string, motif: int, concat: string]

##### now filter motif 4 all

In [33]:
#try to filter motif 5, if not we may have an issue
#read in motif 5
start_time = time.time()
print(start_time)
df_motif_4 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motifs/4").cache()
print("read time all: "+ str(time.time() - start_time))

1681052901.7389646
read time all: 0.4986238479614258


In [34]:
#count dfunpersistf_4
start = time.time()
print(start)
print(df_motif_4.count())
print("df_motif_4 count: "+ str(time.time() - start))

1681052902.2424214


439525168
df_motif_4 count: 205.62926268577576


In [35]:
#filter motif 5
start = time.time()
print(start)
df_motif_4 = df_motif_4.join(df_filter, ["Trace"], "leftanti").cache()
print("df_motif_4 join: "+ str(time.time() - start))

1681053107.8760726
df_motif_4 join: 0.03316521644592285


In [36]:
#count motif 5
start = time.time()
print(start)
print(df_motif_4.count())
print("motif 5 count: "+ str(time.time() - start))

1681053107.9138927


389553016
motif 5 count: 680.4210677146912


In [37]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
df_motif_4.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/filtered/4")
print("write time: "+ str(time.time() - start_time))

1681053788.3395166


write time: 949.3029878139496


In [38]:
df_motif_4.unpersist()
df_filter.unpersist()

DataFrame[Trace: string]

#### Now onto filtering trace 3

In [1]:
%run trace_encode.ipynb
s3_url_trusted = "s3a://sapient-bucket-trusted/"

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b9ee02bc-8baa-4776-ad89-2c2d8ec19af6;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 266ms :: artifacts dl 6ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/09 15:50:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 15:50:50 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
#I think I should read in the partial first.
start_time = time.time()
print(start_time)
df_int_4 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint/pivint4").cache()
print("read time all: "+ str(time.time() - start_time))

1681055622.52179


read time all: 6.610005855560303


In [3]:
start_time = time.time()
print(start_time)
#now count everything you read in
print(df_int_4.count())
print("read time all: "+ str(time.time() - start_time))

1681055631.5159247


238107952
read time all: 113.91786861419678


In [8]:
#quick peek at what the data looks like 
df_int_4.limit(2).toPandas()

,Trace,e1_first(src),e1_first(dst),e2_first(dst),e3_first(dst),motif,concat
0,1005022347472-4,08a54ea7-2486-489b-b576-ace858b12d12,36c8e58b-a693-465f-9ebb-c05d0383aea6,743dc0ee-ab92-4e70-8add-bf533bfc233f,7a0da822-d7fd-4705-9119-43a42d24f56e,4,08a54ea7-2486-489b-b576-ace858b12d1236c8e58b-a...
1,1005022347574-4,00fb243e-bec7-4fb4-b133-9b337db5557b,6257db09-fa0f-4160-bc17-97f22adc951c,743dc0ee-ab92-4e70-8add-bf533bfc233f,7a0da822-d7fd-4705-9119-43a42d24f56e,4,00fb243e-bec7-4fb4-b133-9b337db5557b6257db09-f...


In [7]:
#first drop the last column and the current motif
df_int_4 = df_int_4.drop('e4_first(dst)').cache()

In [9]:
df_int_4.select('motif').distinct().show()

+-----+
|motif|
+-----+
|    4|
|    3|
|    2|
+-----+



In [10]:
start = time.time()
print(start)
#concat columns and compare to drop
df_int_4 = df_int_4.withColumn('concat',concat(df_int_4.columns[1], df_int_4.columns[2], 
                                     df_int_4.columns[3], df_int_4.columns[4])).cache()
print("int 4 concat: "+ str(time.time() - start))

1681056140.2651806
int 4 concat: 0.05903029441833496


In [11]:
start_time = time.time()
#now count everything you read in
print(df_int_4.count())
print("read time all: "+ str(time.time() - start_time))

238107952
read time all: 144.27227783203125


In [12]:
start = time.time()
print(start)
#now set the window. partition by concat and order by concat all for int mat, pull inv filter
#if this step is too hot to handle then you can revert back to simply select columns etc
window = Window.partitionBy("concat").orderBy(col("motif").desc())
col_int = df_int_4.orderBy(col('concat'))\
            .withColumn("rank", rank().over(window))\
                .filter((col("rank") != 1) & (col('concat').isNotNull()))\
                .select('Trace').distinct().collect()
print("int 4 col gen: "+ str(time.time() - start))

1681056290.0372102


int 4 col gen: 149.01053094863892


In [13]:
#create df for anti-join
start = time.time()
print(start)
#turn the col_int into a dataframe
from pyspark.sql import Row
R = Row('Trace')
# use the row function to create a bunch of rows
df_filter = spark.createDataFrame([R(x.Trace) for x in col_int]).cache()
print("dataframe create: "+ str(time.time() - start))

1681056439.0523725
dataframe create: 212.71369338035583


In [14]:
#count for df_int
start = time.time()
print(start)
print(df_filter.count())
print(len(col_int[0]))
print("df_int count: "+ str(time.time() - start))

1681056651.7709193


17308235
1
df_int count: 4.661752223968506


In [15]:
#write the int dataframe right now
start_time = time.time()
print(start_time)
df_filter.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/filters_all/filter_4")
print("write time: "+ str(time.time() - start_time))

1681056656.4372032


write time: 35.15904378890991


In [16]:
#clear col_int
df_filter.unpersist()
col_int.clear()

In [19]:
#now read in filter list, and then read in motif 5 data
start_time = time.time()
print(start_time)
df_filter = spark.read.parquet(f"{s3_url_trusted}/prod/graph/filters_all/filter_4").cache()
print("read time all: "+ str(time.time() - start_time))

1681056789.2313719
read time all: 0.30702805519104004


In [20]:
#count filter
start = time.time()
print(start)
print(df_filter.count())
print("df_int count: "+ str(time.time() - start))

1681056793.3436227


17308235
df_int count: 2.88210129737854


In [21]:
df_filter.limit(5).show()

+---------------+
|          Trace|
+---------------+
|   8590410322-3|
| 575526087924-3|
|1331439993799-3|
|  17180364305-3|
| 420907511591-3|
+---------------+



##### Now work on pivint4

In [22]:
#before you read in df_5 you need to filter the df_int matrix
#first separate the int dfs into motifs - 1 and all other motifs
df_int_a = df_int_4.filter(col("motif") == 3).cache()

In [23]:
#count df_int_a
start = time.time()
print(start)
print(df_int_a.count())
print("df_int_a count: "+ str(time.time() - start))

1681056848.107922


110105995
df_int_a count: 103.91281366348267


In [24]:
df_int_b = df_int_4.filter(col("motif") < 3).cache()

In [25]:
#count df_int_b
start = time.time()
print(start)
print(df_int_b.count())
print("df_int_a count: "+ str(time.time() - start))

1681056952.044404


30613703
df_int_a count: 45.017478942871094


In [26]:
df_int_4.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, motif: int, concat: string]

In [27]:
#filter int_a
#now execute anti join
start = time.time()
print(start)
df_int_a = df_int_a.join(df_filter, ["Trace"], "leftanti").cache()
print("antijoin: "+ str(time.time() - start))

1681056997.1019576
antijoin: 0.07093667984008789


In [28]:
#count a 
#count df_int_a
start = time.time()
print(start)
print(df_int_a.count())
print("df_int_a count: "+ str(time.time() - start))

1681056997.1772232


92797760
df_int_a count: 62.310394048690796


In [29]:
#union all the ints, select on motif 
df_int_3 = df_int_a.union(df_int_b).cache()

In [30]:
#count df_int_a
start = time.time()
print(start)
print(df_int_3.count())
print("df_int_3 count: "+ str(time.time() - start))

1681057059.5383801


123411463
df_int_3 count: 85.40104174613953


In [31]:
df_int_a.unpersist()
df_int_b.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, motif: int, concat: string]

In [32]:
start_time = time.time()
df_int_3.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint/pivint3")
print("write time: "+ str(time.time() - start_time))

write time: 494.2364308834076


In [33]:
df_int_3.unpersist()

DataFrame[Trace: string, e1_first(src): string, e1_first(dst): string, e2_first(dst): string, e3_first(dst): string, motif: int, concat: string]

##### now filter motif 3

In [34]:
#try to filter motif 5, if not we may have an issue
#read in motif 5
start_time = time.time()
print(start_time)
df_motif_3 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motifs/3").cache()
print("read time all: "+ str(time.time() - start_time))

1681057639.2233677
read time all: 0.457775354385376


In [35]:
#count dfunpersistf_4
start = time.time()
print(start)
print(df_motif_3.count())
print("df_motif_3 count: "+ str(time.time() - start))

1681057639.685428


330317985
df_motif_3 count: 167.5418312549591


In [36]:
#filter motif 5
start = time.time()
print(start)
df_motif_3 = df_motif_3.join(df_filter, ["Trace"], "leftanti").cache()
print("df_motif_3 join: "+ str(time.time() - start))

1681057807.2315397
df_motif_3 join: 0.03256702423095703


In [37]:
#count motif 5
start = time.time()
print(start)
print(df_motif_3.count())
print("motif 3 count: "+ str(time.time() - start))

1681057807.2682076


278393280
motif 3 count: 219.56270575523376


In [38]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
df_motif_3.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/filtered/3")
print("write time: "+ str(time.time() - start_time))

1681058026.8357575


write time: 701.4553935527802


In [39]:
df_motif_3.unpersist()
df_filter.unpersist()

DataFrame[Trace: string]

#### Now do everything for trace 2!!

In [1]:
%run trace_encode.ipynb
s3_url_trusted = "s3a://sapient-bucket-trusted/"

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ac33e6c5-459d-4ab8-9ec1-7c85b803d786;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 157ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/09 16:48:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 16:48:49 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
#I think I should read in the partial first.
start_time = time.time()
print(start_time)
df_int_3 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/all_motifs/pivint/pivint3").cache()
print("read time all: "+ str(time.time() - start_time))

1681058935.9945955


read time all: 6.843519687652588


In [3]:
start_time = time.time()
print(start_time)
#now count everything you read in
print(df_int_3.count())
print("read time all: "+ str(time.time() - start_time))

1681058942.8426425


123411463
read time all: 52.45708394050598


In [5]:
#quick peek at what the data looks like 
df_int_3.limit(2).toPandas()

,Trace,e1_first(src),e1_first(dst),e2_first(dst),motif,concat
0,1005022347312-3,a07fa153-c9f9-4c37-bd0c-ae41ff700f29,d47bfdd9-8b11-4fe5-bd8c-4fe8071140af,605ee76d-8fa6-4bf8-92da-8dcab1c33b43,3,a07fa153-c9f9-4c37-bd0c-ae41ff700f29d47bfdd9-8...
1,1005022347353-3,282bfbdc-aafe-4326-b157-22643db6cd04,974bcc72-fb40-4907-b9e8-174954644deb,05597551-552e-4e4a-b888-2be93559b617,3,282bfbdc-aafe-4326-b157-22643db6cd04974bcc72-f...


In [4]:
#first drop the last column and the current motif
df_int_3 = df_int_3.drop('e3_first(dst)').cache()

In [6]:
df_int_3.select('motif').distinct().show()

+-----+
|motif|
+-----+
|    3|
|    2|
+-----+



In [7]:
start = time.time()
print(start)
#concat columns and compare to drop
df_int_3 = df_int_3.withColumn('concat',concat(df_int_3.columns[1], df_int_3.columns[2], 
                                     df_int_3.columns[3])).cache()
print("int 3 concat: "+ str(time.time() - start))

1681059139.3101964
int 3 concat: 0.054877281188964844


In [8]:
start_time = time.time()
#now count everything you read in
print(df_int_3.count())
print("read time all: "+ str(time.time() - start_time))

123411463
read time all: 80.91670417785645


In [9]:
start = time.time()
print(start)
#now set the window. partition by concat and order by concat all for int mat, pull inv filter
#if this step is too hot to handle then you can revert back to simply select columns etc
window = Window.partitionBy("concat").orderBy(col("motif").desc())
col_int = df_int_3.orderBy(col('concat'))\
            .withColumn("rank", rank().over(window))\
                .filter((col("rank") != 1) & (col('concat').isNotNull()))\
                .select('Trace').distinct().collect()
print("int 3 col gen: "+ str(time.time() - start))

1681059224.4690127


int 3 col gen: 60.258606910705566


In [10]:
#create df for anti-join
start = time.time()
print(start)
#turn the col_int into a dataframe
from pyspark.sql import Row
R = Row('Trace')
# use the row function to create a bunch of rows
df_filter = spark.createDataFrame([R(x.Trace) for x in col_int]).cache()
print("dataframe create: "+ str(time.time() - start))

1681059284.7325149
dataframe create: 91.74432229995728


In [11]:
#count for df_int
start = time.time()
print(start)
print(df_filter.count())
print("df_int count: "+ str(time.time() - start))

1681059376.481606


7365254
df_int count: 1.8421056270599365


In [12]:
#write the int dataframe right now
start_time = time.time()
print(start_time)
df_filter.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/filters_all/filter_3")
print("write time: "+ str(time.time() - start_time))

1681059378.328299


write time: 20.023906469345093


In [13]:
#clear col_int
df_filter.unpersist()
col_int.clear()

In [14]:
#now read in filter list, and then read in motif 5 data
start_time = time.time()
print(start_time)
df_filter = spark.read.parquet(f"{s3_url_trusted}/prod/graph/filters_all/filter_3").cache()
print("read time all: "+ str(time.time() - start_time))

1681059398.9610183
read time all: 0.21618032455444336


In [15]:
#count filter
start = time.time()
print(start)
print(df_filter.count())
print("df_int count: "+ str(time.time() - start))

1681059399.1816046
7365254
df_int count: 0.860105037689209


In [16]:
df_filter.limit(5).show()

+---------------+
|          Trace|
+---------------+
| 781684082668-2|
| 283467852112-2|
| 661425042903-2|
| 429496770170-2|
|1176821133542-2|
+---------------+



##### now filter motif 2

In [17]:
#try to filter motif 5, if not we may have an issue
#read in motif 5
start_time = time.time()
print(start_time)
df_motif_2 = spark.read.parquet(f"{s3_url_trusted}/prod/graph/motifs/2").cache()
print("read time all: "+ str(time.time() - start_time))

1681059497.1049292
read time all: 0.44818592071533203


In [18]:
#count
start = time.time()
print(start)
print(df_motif_2.count())
print("df_motif_2 count: "+ str(time.time() - start))

1681059500.9240417


61227406
df_motif_2 count: 34.9476113319397


In [19]:
#filter motif 5
start = time.time()
print(start)
df_motif_2 = df_motif_2.join(df_filter, ["Trace"], "leftanti").cache()
print("df_motif_2 join: "+ str(time.time() - start))

1681059535.8761902
df_motif_2 join: 0.07049059867858887


In [20]:
#count motif 5
start = time.time()
print(start)
print(df_motif_2.count())
print("motif 2 count: "+ str(time.time() - start))

1681059535.9509368


46496898
motif 2 count: 28.631112575531006


In [21]:
#write to parquet and unpersist
start_time = time.time()
print(start_time)
df_motif_2.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/motifs/filtered/2")
print("write time: "+ str(time.time() - start_time))

1681059564.5869417


write time: 182.24507784843445


In [22]:
df_motif_2.unpersist()
df_filter.unpersist()

DataFrame[Trace: string]

### Now one hot encoding! 

In [26]:
start_time = time.time()
#instantiate dictionary and return df
dict_mapping = {}
#list of sparse cols
list_sparse = []
#attempting one hot without string indexer
df_all_motifs, dict_mapping, list_sparse = oneHotCol(df_all_motifs,'object', dict_mapping, list_sparse)
print("object onehot time: "+ str(time.time() - start_time))

TypeError: Invalid param value given for param "inputCols". Could not convert object to list of strings

In [ ]:
df_all_motifs.limit(2).toPandas()

In [ ]:
#now write results
start_time = time.time()
df_all_motifs.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all/onehot")
print("write time: "+ str(time.time() - start_time))

In [ ]:
#now unpersist
df_all_motifs.unpersist()

In [ ]:
#and read back in
start_time = time.time()
df_all_motifs = spark.read.parquet(f"{s3_url_trusted}/prod/graph/all/onehot").cache()
print("read time all: "+ str(time.time() - start_time))

In [ ]:
#and count 
start_time = time.time()
print(df_all_motifs.count())
print("count : "+ str(time.time() - start_time))

In [ ]:
#and count 
start_time = time.time()
print(df_all_motifs.count())
print("count : "+ str(time.time() - start_time))

In [ ]:
dict_mapping

In [ ]:
list_cols = ['file_path_ext','image_path',
             'parent_image_path',
             'action','timestamp_bins']

In [ ]:
#then run for c in list cols above, one hot encode...then write

In [26]:
start_time = time.time()
df_all_motifs = df_2.union(df_3).union(df_4).union(df_5).union(df_6).cache()
print("union: "+ str(time.time() - start_time))

union: 0.20996594429016113


In [27]:
start_time = time.time()
print(df_all_motifs.count())
print("all count: "+ str(time.time() - start_time))

23/04/06 02:37:20 ERROR DiskBlockObjectWriter: Exception occurred while manually close the output stream to file /home/ec2-user/SageMaker/tmp/blockmgr-e4790cc7-50bb-4348-ba56-1bc5e220124d/3b/temp_shuffle_e4374012-b3bd-447a-bc17-4a0116768cf3, No space left on device
23/04/06 02:37:20 ERROR DiskBlockObjectWriter: Exception occurred while manually close the output stream to file /home/ec2-user/SageMaker/tmp/blockmgr-e4790cc7-50bb-4348-ba56-1bc5e220124d/0f/temp_shuffle_81f49ba8-fdc7-4642-af17-d6db3bf38dfb, No space left on device
23/04/06 02:37:20 ERROR DiskBlockObjectWriter: Exception occurred while manually close the output stream to file /home/ec2-user/SageMaker/tmp/blockmgr-e4790cc7-50bb-4348-ba56-1bc5e220124d/36/temp_shuffle_753e6fba-5c74-45ae-b60e-400c5a756ff2, No space left on device
23/04/06 02:37:20 ERROR DiskBlockObjectWriter: Exception occurred while manually close the output stream to file /home/ec2-user/SageMaker/tmp/blockmgr-e4790cc7-50bb-4348-ba56-1bc5e220124d/26/temp_shuffl

Py4JJavaError: An error occurred while calling o876.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 606.0 failed 1 times, most recent failure: Lost task 10.0 in stage 606.0 (TID 25697) (ip-172-16-94-2.us-west-2.compute.internal executor driver): java.io.IOException: No space left on device
	at java.io.FileOutputStream.writeBytes(Native Method)
	at java.io.FileOutputStream.write(FileOutputStream.java:326)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at org.apache.spark.io.MutableCheckedOutputStream.write(MutableCheckedOutputStream.scala:43)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:126)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:225)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:178)
	at java.io.BufferedOutputStream.flushBuffer(B

In [ ]:
df_2.unpersist()
df_3.unpersist()
df_4.unpersist()
df_5.unpersist()
df_6.unpersist()

In [ ]:
df_all_motifs.printSchema()

In [ ]:
#make a motif column to partition by motifs


In [15]:
df_all_motifs.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(f"{s3_url_trusted}/prod/graph/all_motifs")

23/04/04 23:35:42 ERROR Executor: Exception in task 767.0 in stage 109.0 (TID 11313)
java.io.IOException: No space left on device
	at sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:60)
	at sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:93)
	at sun.nio.ch.IOUtil.write(IOUtil.java:65)
	at sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:211)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:353)
	at java.nio.channels.Channels.writeFullyImpl(Channels.java:78)
	at java.nio.channels.Channels.writeFully(Channels.java:101)
	at java.nio.channels.Channels.access$000(Channels.java:61)
	at java.nio.channels.Channels$1.write(Channels.java:174)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:225)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:178)
	at java.io.Object

23/04/04 23:35:42 ERROR FileFormatWriter: Aborting job 92c80e86-3d03-47d9-9101-17470447aff8.
org.apache.spark.SparkException: Job aborted due to stage failure: Task 767 in stage 109.0 failed 1 times, most recent failure: Lost task 767.0 in stage 109.0 (TID 11313) (ip-172-16-94-2.us-west-2.compute.internal executor driver): java.io.IOException: No space left on device
	at sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:60)
	at sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:93)
	at sun.nio.ch.IOUtil.write(IOUtil.java:65)
	at sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:211)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:353)
	at java.nio.channels.Channels.writeFullyImpl(Channels.java:78)
	at java.nio.channels.Channels.writeFully(Channels.java:101)
	at java.nio.channels.Channels.access$000(Channels.java:61)
	at java.nio.channels.Channels$1.write(Channels.java:174)
	at java.io.Buff

Py4JJavaError: An error occurred while calling o747.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org

In [ ]:
df_all_motifs.unpersist()

In [ ]:
start_time = time.time()
df_all_motifs = spark.read.parquet(f"{s3_url_trusted}/prod/graph/all_motifs").cache()
print("read all_motifs: "+ str(time.time() - start_time))

In [ ]:
df_all_motifs.count()

In [ ]:
%run trace_encode.ipynb

In [ ]:
#instantiate dictionary and return df
dict_mapping = {}
#list of sparse cols
list_sparse = []

df_all_motifs, dict_mapping, list_sparse = oneHotCol(df_all_motifs,'object', dict_mapping, list_sparse)

In [ ]:
print(dict_mapping)

In [ ]:
start_time = time.time()
df_onehot.write.option("maxRecordsPerFile", 300000).mode("overwrite").partitionBy("hostname").parquet(f"{s3_url_trusted}/prod/graph/onehot_traces")
df_onehot.unpersist()
print("write onehot: "+ str(time.time() - start_time))

In [ ]:
# single function 
df_onehot = oneHotCol(df_all)...pivot_ind()

In [ ]:
df_onehot.write.option("maxRecordsPerFile", 300000).mode("overwrite").partitionBy("event_day", "event_hour").parquet(f"{s3_url_trusted}/prod/graph/onehot_traces")

In [ ]:
# to do: see if duplicate subpaths can be removed